In [ ]:
!pip install vfb-connect --upgrade

### A note on using these notebooks
This is designed as an interactive tutorial. Feel free to add code cells below each example to try out variations of your own.

In [ ]:
# Import libs and initialise API objects
from vfb_connect.cross_server_tools import VfbConnect
import pandas as pd

vc = VfbConnect()

import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')

# Many functions return JSON-compatible nested data structures. This function coverts them to DataFrame.
def summary_2_df(summary, sort=None):
    """Convert summary to DataFrame.  Optionally specify a set of columns to sort as a list of strings"""
    if sort:
        return pd.DataFrame.from_records(summary).sort_values(sort)
    else:
        return pd.DataFrame.from_records(summary)

## Mapping

Use case: If I have a single neuron, how can I find other neurons of the same or similar type within or between data sources?

#### Mapping via common parent type



In [ ]:
# lets take some examples from a discovery query on the previous spreadhseet
sayin_tab = pd.DataFrame.from_records(vc.get_instances_by_dataset('Sayin2019', summary=True))
sayin_tab

In [ ]:
oct_VPM3 = summary_2_df(vc.get_instances('octopaminergic VPM3 neuron', summary=True))
oct_VPM3

In [ ]:
oct_VPM3_images = vc.neo_query_wrapper.get_images(oct_VPM3['id'], stomp=True, template='JRC2018Unisex', image_folder = 'oct_VPM3b')
oct_VPM3_images

In [ ]:
oct_VPM3_images = vc.get_images_by_type('octopaminergic VPM3 neuron', stomp=True, template='JRC2018Unisex', image_folder = 'oct_VPM3')
nl = navis.read_swc('./oct_VPM3')
navis.plot3d(nl)

#### Mapping via NBLAST

VFB supports image based queries that find neurons with similar morphology to a query neuron. We use a matrix of [NBLAST](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4961245/) scores for this purpose.  This currently covers mapping between FAFB & Hemibrain and internal mappings within the hemibrain and FLyCircuit neurons. More mappings will be added soon, as well as support for other similarity mappings.

In [ ]:
summary_2_df(vc.neo_query_wrapper.get_anatomical_individual_TermInfo(['VFB_001012ay'], summary=True))

In [ ]:
similar_to_EPG6L1 = vc.get_similar_neurons('VFB_001012ay')
similar_to_EPG6L1 

In [ ]:
EPG6L1_similar_images = vc.neo_query_wrapper.get_images(list(similar_to_EPG6L1['id']) + ['VFB_001012ay'], 
                                template='JRC2018Unisex',
                                image_folder='EPG6L1',
                                stomp=True) # Note - stomp allows an existing folder to be over-written


EPG6L1_similar_images


In [ ]:
list(EPG6L1_similar_images['parents_label'])

In [ ]:
nl = navis.read_swc('./EPG6L1')
nl.plot3d(hover_name=True)

In [ ]:
## Generate a link to display these neurons on VFB

print(vc.get_vfb_link(list(similar_to_EPG6L1['id']) + ['VFB_001012ay'], template='JRC2018Unisex'))


Note: to clean up 'spiky' neurons with obvious alignment issues you can use the navis function [despike](https://navis.readthedocs.io/en/latest/source/tutorials/generated/navis.despike_neuron.html#navis.despike_neuron)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=94cb7c07-bfd7-4142-b5f9-bec41746df04' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>